### Importacion de librerias

In [12]:
### Importacion de archivos
import pandas as pd
import numpy as np

### Carga de archivos

In [13]:
clientes = pd.read_csv('current_reports/clientes.csv')
df_clientes = pd.DataFrame(clientes)

### Manipulación de reporte de clientes

In [14]:
df_clientes = df_clientes.drop(df_clientes[df_clientes["Sucursal"] == "Sucursal Coyoacan"].index)
df_clientes.rename(columns={'index': "BN-Cliente",'Apellido': 'Apellidos', 'pos_client': "POS-Cliente", 'reg_dtt': 'F/h registro',
                            'client_type': 'Tipo', 'email': 'Correo', 'is_active': 'Web', 'region': 'Region',
                            'lamin_discount': 'Desc Laminado', 'steel_discount': 'Desc Acero', 'buy_back_enabled': 'Re-compra',
                            'csg_enabled': 'Consigna', 'phone': 'Teléfono', 'csg_days':'Dias de Consigna', 'csg_auth':'Consigna Autorizada',
                            'accumulated_purchases': 'Acumulado 6m'}, inplace=True)


# Rellenando datos faltantes
df_clientes['Nombre'] = df_clientes['Nombre'].fillna('<NULL>')
df_clientes['Apellidos'] = df_clientes['Apellidos'].fillna('<NULL>')
df_clientes['name'] = df_clientes['name'].fillna('<NULL>')
df_clientes['Dias de Consigna'] = df_clientes['Dias de Consigna'].fillna('<NULL>')
df_clientes['POS-Cliente'] = df_clientes['POS-Cliente'].fillna('<NULL>')
df_clientes['Correo'] = df_clientes['Correo'].fillna('<NULL>')
df_clientes['Nombre'] = df_clientes['Nombre'].map(lambda x: "" if ( x == "<NULL>" ) else x)
df_clientes['Apellidos'] = df_clientes['Apellidos'].map(lambda x: "" if ( x == "<NULL>" ) else x)
df_clientes['name'] = df_clientes['name'].map(lambda x: "" if ( x == "<NULL>" ) else x)
df_clientes['Dias de Consigna'] = df_clientes['Dias de Consigna'].map(lambda x: 0 if ( x == "<NULL>" ) else x)
df_clientes['Acumulado 6m'] = df_clientes['Acumulado 6m'].map(lambda x: 0 if ( x == "<NULL>" ) else x)

# Definiendo tipos de dato
df_clientes['POS-Cliente'] = df_clientes['POS-Cliente'].astype('string')
df_clientes['Tipo'] = df_clientes['Tipo'].astype('string')
df_clientes['Nombre'] = df_clientes['Nombre'].astype('string')
df_clientes['Apellidos'] = df_clientes['Apellidos'].astype('string')
df_clientes['Vendedor Asignado'] = df_clientes['Vendedor Asignado'].astype('string')
df_clientes['Correo'] = df_clientes['Correo'].astype('string')
df_clientes['Web'] = df_clientes['Web'].astype('bool')
df_clientes['Dias de Consigna'] = df_clientes['Dias de Consigna'].astype('float64')
df_clientes['Acumulado 6m'] = df_clientes['Acumulado 6m'].astype('float64')
df_clientes['Teléfono'] = df_clientes['Teléfono'].astype('string')
df_clientes['Region'] = df_clientes['Region'].astype('string')
df_clientes['Sucursal'] = df_clientes['Sucursal'].astype('string')
df_clientes['Re-compra'] = df_clientes['Re-compra'].astype('bool')
df_clientes['Consigna'] = df_clientes['Consigna'].astype('bool')


### Unificación de nombres de cliente

In [15]:
# Formateando nombres
df_clientes['Nombre'] = np.where(df_clientes['Nombre'] == '', df_clientes['name'], df_clientes['Nombre']) 
df_clientes['Nombre Completo'] = df_clientes['Nombre'] + " " + df_clientes['Apellidos']
df_clientes['Nombre Completo'] = np.where(df_clientes['Nombre Completo'] == ' ', df_clientes['name'], df_clientes['Nombre Completo']) 
df_clientes['Nombre Completo'] = df_clientes['Nombre Completo'].map(lambda x: x if x[:len(x)] == " " else x.rstrip())
df_clientes['Nombre Completo'] = np.where(df_clientes['Nombre Completo'] == '', None, df_clientes['Nombre Completo']) 
df_clientes['Apellidos'] = df_clientes['Apellidos'].str.upper()
df_clientes['Nombre'] = df_clientes['Nombre'].str.upper()
df_clientes['Nombre Completo'] = df_clientes['Nombre Completo'].str.upper()
df_clientes['name'] = df_clientes['name'].str.upper()

# Archivo para unificacion de nombres de clientes
with pd.ExcelWriter('utils/unificacionNombres.xlsx') as writer:
    df_clientes[["BN-Cliente", "Nombre Completo", "Nombre", "Apellidos"]].to_excel(writer, index=False)

### Desgloce de clientes sin numero telefonico

In [16]:
df_clientes['Teléfono'] = df_clientes['Teléfono'].fillna('<NULL>')
def valid_phone(phone):
    count = {}
    previous_c = ""
    for c in phone:
        if c == previous_c:
            if c in count:
                count[c] += 1
            else:
                count[c] = 1
        previous_c = c
            
    for key in count:
        if count[key] >= 5:
            return "<NULL>"
    
    if len(phone) == 10: 
        return phone
    else:
        return "<NULL>"
    
    
df_clientes['Teléfono'] = df_clientes['Teléfono'].map(lambda x: valid_phone(x))
df_clientes_not_phone = df_clientes[df_clientes['Teléfono'] == "<NULL>"]
df_clientes_not_phone = df_clientes_not_phone[['BN-Cliente', 'POS-Cliente', 'F/h registro', 'Nombre Completo', 'Correo', 
                                               'Teléfono', 'Region', 'Sucursal']]

# Archivo de clientes sin numero o numero invalido
with pd.ExcelWriter('utils/telefonos_invalidos.xlsx') as writer:
    df_clientes_not_phone.to_excel(writer, index=False)

df_clientes_not_phone['BN-Cliente'] = np.where(df_clientes_not_phone['POS-Cliente'] == '<NULL>', "Si", "No")
df_clientes_not_phone['POS-Cliente'] = np.where(df_clientes_not_phone['POS-Cliente'] == '<NULL>', "No", "Si")
df_clientes_not_phone['Correo'] = np.where(df_clientes_not_phone['Correo'] == '<NULL>', 'No', 'Si')
df_clientes_not_phone['Nombre Completo'] = np.where(df_clientes_not_phone['Nombre Completo'] == '', "No", "Si")
count_correo = df_clientes_not_phone['Correo'].value_counts()
count_pos = df_clientes_not_phone['POS-Cliente'].value_counts()
count_bn = df_clientes_not_phone['BN-Cliente'].value_counts()
count_name = df_clientes_not_phone['Nombre Completo'].value_counts()
count_region = df_clientes_not_phone['Region'].value_counts()
df_resumen = pd.DataFrame([count_name, count_correo, count_bn, count_pos, count_region]).fillna(0)
df_resumen

,Si,No,CDMX,GDL,<NULL>
Nombre Completo,1699.0,0.0,0.0,0.0,0.0
Correo,83.0,1616.0,0.0,0.0,0.0
BN-Cliente,82.0,1617.0,0.0,0.0,0.0
POS-Cliente,1617.0,82.0,0.0,0.0,0.0
Region,0.0,0.0,1444.0,234.0,21.0


### Reporte clientes prospecto web

In [17]:
df_clientes_web = df_clientes[df_clientes['Nombre Completo'].isnull()]
df_clientes_web = df_clientes_web[df_clientes_web['Agregado por'] == "<NULL>"]
df_clientes_web = df_clientes_web[['BN-Cliente', 'F/h registro', 'Vendedor Asignado', 'username', 'Correo', 'Teléfono', 'Region', 'Sucursal']]

df_clientes_web_resumen = df_clientes_web[['BN-Cliente', 'F/h registro', 'username', 'Correo', 'Teléfono', 'Vendedor Asignado', 'Region', 'Sucursal']]
df_clientes_web_resumen['username'] = np.where(df_clientes_web_resumen['username'] == '<NULL>', "No", "Si")
df_clientes_web_resumen['Correo'] = np.where(df_clientes_web_resumen['Correo'] == '<NULL>', "No", "Si")
df_clientes_web_resumen['Teléfono'] = np.where(df_clientes_web_resumen['Teléfono'] == '<NULL>', "No", "Si")
df_clientes_web_resumen['Se puede contactar'] = np.where((df_clientes_web_resumen['Teléfono'] == 'No') & (df_clientes_web_resumen['Correo'] == 'No'), "No", "Si")

count_correo = df_clientes_web_resumen['Correo'].value_counts()
count_phone = df_clientes_web_resumen['Teléfono'].value_counts()
count_username = df_clientes_web_resumen['username'].value_counts()
count_region = df_clientes_web_resumen['Region'].value_counts()
count_aviable = df_clientes_web_resumen['Se puede contactar'].value_counts()

df_clientes_web_resumen = df_clientes_web_resumen[['BN-Cliente', 'Se puede contactar']]
df_clientes_web = df_clientes_web.merge(df_clientes_web_resumen, how='left', on='BN-Cliente')
df_clientes_web = df_clientes_web.drop(df_clientes_web[df_clientes_web["Se puede contactar"] == "No"].index)

# Archivo de clientes sin numero o numero invalido
with pd.ExcelWriter('new_reports/clientes_prospecto_web.xlsx') as writer:
    df_clientes_web[['username', 'Correo', 'Teléfono', 'Region']].to_excel(writer, index=False)

df_resumen = pd.DataFrame([count_aviable, count_username, count_phone, count_correo, count_region]).fillna(0)
df_resumen


,Si,No,<NULL>,CDMX
Se puede contactar,158.0,5.0,0.0,0.0
username,161.0,2.0,0.0,0.0
Teléfono,137.0,26.0,0.0,0.0
Correo,148.0,15.0,0.0,0.0
Region,0.0,0.0,146.0,17.0


### Reporte Expedientes

In [18]:
df_expedientes = df_clientes[['BN-Cliente', 'POS-Cliente', 'Nombre Completo', 'Correo', 'Teléfono', 'Region', 'Sucursal',
                              'Vendedor Asignado', 'Desc Acero', 'Desc Laminado', 'Re-compra', 'Dias de Consigna', 'Consigna Autorizada', 'F/h registro']]

# # Archivo de clientes sin numero o numero invalido
with pd.ExcelWriter('new_reports/expediente_clientes.xlsx') as writer:
    df_expedientes.to_excel(writer, index=False)

### Reporte Estadisticas de compra

In [19]:
df_clientes.columns
df_estadisticas_compra = df_clientes[['BN-Cliente', 'POS-Cliente', 'Nombre Completo','Vendedor Asignado', 'Desc Acero', 
                                      'Desc Laminado', 'Acumulado 6m']]
df_estadisticas_compra.head()
# Archivo de clientes sin numero o numero invalido
# with pd.ExcelWriter('new_reports/estadisticas_compra.xlsx') as writer:
#     df_estadisticas_compra.to_excel(writer, index=False)

,BN-Cliente,POS-Cliente,Nombre Completo,Vendedor Asignado,Desc Acero,Desc Laminado,Acumulado 6m
0,1,1,CLIENTE MOSTRADOR,-,0.0,10.0,0.00
1,2,<NULL>,INVITADO E-STORE,-,10.0,10.0,0.00
2,3,2297,MARIA DEL ROCIO JUAREZ ARAGON,Ricardo,60.0,60.0,187929.70
3,4,2624,LAURA FERNANDEZ FLORES,Diana Aidee,57.5,60.0,125577.13
4,5,567,GEMA / MARIA ISABEL GOMEZ,Patricia,55.0,60.0,65000.69
